In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import root_mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from catboost import CatBoostRegressor 

In [2]:
train_data = pd.read_csv("C:/Users/czvzxv/Desktop/Такси/train_model_data.csv")
test_data = pd.read_csv("C:/Users/czvzxv/Desktop/Такси/test_model_data.csv")

In [3]:
train_data.dtypes

Unnamed: 0              int64
vendor_id               int64
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
distance_km           float64
speed_kmh             float64
month                   int64
hour                    int64
wday                    int64
low_speed_zone          int64
bearing               float64
jfk_dist_pick         float64
jfk_dist_drop         float64
lg_dist_pick          float64
lg_dist_drop          float64
jfk_trip                int64
lg_trip                 int64
blizzard                int64
rain                  float64
s_fall                float64
all_precip            float64
has_snow                int64
has_rain                int64
s_depth               float64
max_temp                int64
min_temp                int64
dtype: object

In [4]:
train_data = train_data.drop(['Unnamed: 0','speed_kmh'],axis = 1)
test_data = test_data.drop(['Unnamed: 0','speed_kmh','dropoff_datetime','trip_duration'],axis = 1)

In [5]:
train_data['store_and_fwd_flag'] = train_data['store_and_fwd_flag'].replace({'Y': 1, 'N': 0})
test_data['store_and_fwd_flag'] = test_data['store_and_fwd_flag'].replace({'Y': 1, 'N': 0})

In [6]:
y = train_data['trip_duration']
X = train_data.drop(['trip_duration'],axis = 1)

In [7]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,shuffle=True,random_state=42)

In [8]:
model_lin_reg = LinearRegression(fit_intercept = True).fit(X_train,y_train)
y_pred = model_lin_reg.predict(X_test)
y_pred = np.maximum(y_pred, 0)

In [9]:
print('RMSLE: ', root_mean_squared_log_error(y_test,y_pred))

RMSLE:  0.5026439805423633


In [10]:
y_test_lin_pred = model_lin_reg.predict(test_data)
y_test_lin_pred = np.maximum(y_test_lin_pred , 0)

In [11]:
sub_file = pd.read_csv("C:/Users/czvzxv/Desktop/Такси/sample_submission.csv")
sub_file['trip_duration'] = y_test_lin_pred
output_file = "submission.csv"
sub_file.to_csv("C:/Users/czvzxv/Downloads/predictions.csv", index=False)

# Catboost

In [12]:
model_catboost = CatBoostRegressor().fit(X_train,y_train)
y_pred_catboost = model_catboost.predict(X_test)
y_pred_catboost = np.maximum(y_pred_catboost, 0)

Learning rate set to 0.124846
0:	learn: 800.4093851	total: 230ms	remaining: 3m 49s
1:	learn: 767.5978680	total: 320ms	remaining: 2m 39s
2:	learn: 740.9747197	total: 428ms	remaining: 2m 22s
3:	learn: 719.4489414	total: 564ms	remaining: 2m 20s
4:	learn: 701.7024405	total: 688ms	remaining: 2m 17s
5:	learn: 687.8397015	total: 805ms	remaining: 2m 13s
6:	learn: 676.6564602	total: 922ms	remaining: 2m 10s
7:	learn: 667.2858315	total: 1.02s	remaining: 2m 6s
8:	learn: 659.7341480	total: 1.12s	remaining: 2m 3s
9:	learn: 653.7939803	total: 1.26s	remaining: 2m 5s
10:	learn: 649.0225881	total: 1.52s	remaining: 2m 16s
11:	learn: 645.1352794	total: 1.73s	remaining: 2m 22s
12:	learn: 641.7517648	total: 1.85s	remaining: 2m 20s
13:	learn: 639.1148702	total: 1.96s	remaining: 2m 17s
14:	learn: 636.9002556	total: 2.07s	remaining: 2m 15s
15:	learn: 634.9096397	total: 2.18s	remaining: 2m 13s
16:	learn: 633.0375881	total: 2.29s	remaining: 2m 12s
17:	learn: 631.5295906	total: 2.41s	remaining: 2m 11s
18:	learn: 

In [13]:
print('RMSLE: ', root_mean_squared_log_error(y_test,y_pred_catboost))

RMSLE:  0.3725085265773666


In [14]:
y_pred_test_catboost = model_catboost.predict(test_data)
y_pred_test_catboost = np.maximum(y_pred_test_catboost, 0)

In [15]:
y_pred_test_catboost

array([ 983.08249291,  620.24925447,  489.37445666, ..., 1565.20312255,
       2068.65686405, 1196.30966472])

In [16]:
sub_file = pd.read_csv("C:/Users/czvzxv/Desktop/Такси/sample_submission.csv")
sub_file['trip_duration'] = y_pred_test_catboost
output_file = "submission.csv"
sub_file.to_csv("C:/Users/czvzxv/Downloads/predictions2.csv", index=False)